In [10]:
#Imports
import urllib3
import os
import requests
import re
from bs4 import BeautifulSoup
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
#Access Main Page
url = 'https://www.imdb.com/chart/top/' #Access imdb TOP 250 List
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
print(r.status_code) # Check for successful connection (200)

200


In [12]:
# Get full urls of TOP 250 List

links=[]
for a in soup.find_all('a'): #, href=True):
    links.append(a.get('href'))

links=['https://www.imdb.com'+a.strip() for a in links if a is not None and a.startswith('/title/tt') ]

#---------------------------Remove duplicates in links
top_250_links=[]
for c in links:
    if c not in top_250_links:
        top_250_links.append(c)

print(len(top_250_links))

250


In [13]:
# Define DF for the data
column_list=['Rank','Movie_name' ,'URL' ,'Release_Year' ,'IMDB_Rating' ,'Metascore' ,
'Reviewer_count' ,'Censor_Board_Rating' ,'Movie_Length' ,'Genre_1' ,
'Genre_2' ,'Genre_3' ,'Genre_4' ,'Release_Date' ,'Story_Summary' ,
'Director' ,'Writer_1' ,'Writer_2' ,'Writer_3' ,'Star_1' ,
'Star_2' ,'Star_3' ,'Star_4' ,'Star_5' ,'Plot_Keywords' ,'Budget' ,
'Gross_USA' ,'Cum_Worldwide_Gross' ,'Production_Company' ]
df = pd.DataFrame(columns=column_list)
df

,Rank,Movie_name,URL,Release_Year,IMDB_Rating,Metascore,Reviewer_count,Censor_Board_Rating,Movie_Length,Genre_1,...,Star_1,Star_2,Star_3,Star_4,Star_5,Plot_Keywords,Budget,Gross_USA,Cum_Worldwide_Gross,Production_Company


In [14]:
# Loop through URLs and retrive data to dataframe (May take a few minutes!)

for x in np.arange(0, len(top_250_links)-1):
    try:

    #---------------------------Load html page for each movie in movies list
        url=top_250_links[x]
        req = requests.get(url)
        page = req.text
        soup = BeautifulSoup(page, 'html.parser')
    
    #---------------------------Retrieve Movie details from html page
        
        Movie_name=(soup.find("div",{"class":"title_wrapper"}).get_text(strip=True).split('|')[0]).split('(')[0]
        
        year_released=((soup.find("div",{"class":"title_wrapper"}).get_text(strip=True).split('|')[0]).split('(')[1]).split(')')[0]
        
        imdb_rating=soup.find("span",{"itemprop":"ratingValue"}).text
        
        metascore=soup.find("div",{"class":"titleReviewBarItem"}).get_text(strip=True).replace("MetascoreFrommetacritic.com","")
    
        reviewer_count=soup.find("span",{"itemprop":"ratingCount"}).text
    
        subtext= soup.find("div",{"class":"subtext"}).get_text(strip=True).split('|') #Censor_rating
        if len(subtext)<4:
            censor_rating='Not Rated'
            movie_len=subtext[0] #Length
            genre_list=subtext[1].split(',') #Genres
            while len(genre_list)<4:         genre_list.append(" ")
            genre_1,genre_2,genre_3,genre_4=genre_list
            release_date=subtext[2] #Release Date
        else:
            censor_rating=subtext[0]
            movie_len=subtext[1]
            genre_list=subtext[2].split(',')
            while len(genre_list)<4:         genre_list.append(" ")
            genre_1,genre_2,genre_3,genre_4=genre_list
            release_date=subtext[3]
        
        #Summary
        story_summary=soup.find("div",{"class":"summary_text"}).get_text(strip=True).strip()
        
        #---------------------------Director,Writer and Actor details
        b=[]
        for a in soup.find_all("div",{"class":"credit_summary_item"}):
            c=re.split(',|:|\|',a.get_text(strip=True)) 
            b.append(c)
        stars=b.pop()
        writers=b.pop()
        directors=b.pop()
        if 'See full cast & crew»' in stars: stars.remove('See full cast & crew»')
        if '1 more credit»' in writers: writers.remove('1 more credit»') 
        if '1 more credit»' in directors: directors.remove('1 more credit»')
        stars=stars[1:]
        writers=writers[1:]
        directors=directors[1:]
        while len(stars)<5:         stars.append(" ")
        while len(writers)<3:         writers.append(" ")
    
        star_1,star_2,star_3,star_4,star_5=stars
        
        writer_1=writers[0]
        writer_2=writers[1]
        writer_3=writers[2]
        
        director=directors[0]
        
        #---------------------------Plot Keywords
        b=[]
        for a in soup.find_all("span",{"class":"itemprop"}):     b.append(a.get_text(strip=True))  
        
        plot_keywords='|'.join(b)
        
        #---------------------------Commercial details and Prod Company
        
        
        b=[]                    #---------------------------Remove unwanted entries
        d={'Budget':'', 'Opening Weekend USA':'','Gross USA':'','Cumulative Worldwide Gross':'','Production Co':''}
        for a in soup.find_all("div",{"class":"txt-block"}):
            c=a.get_text(strip=True).split(':')
            if c[0] in d:
                b.append(c)
    
        for i in b:             #---------------------------Update default values if entries are found
                if i[0] in d: 
                    d.update({i[0]:i[1]})                
    
        production_company=d['Production Co'].split('See more')[0]
        cum_world_gross=d['Cumulative Worldwide Gross'].split(' ')[0]
        gross_usa=d['Gross USA'].split(' ')[0]
        budget=d['Budget']
        
        #---------------------------Dictionary to holds all details
        movie_dict={
            'Rank':x+1,
            'Movie_name' : Movie_name,
            'URL' : url,
            'Release_Year' : year_released,
            'IMDB_Rating' : imdb_rating,
            'Metascore' : metascore,
            'Reviewer_count' : reviewer_count,
            'Censor_Board_Rating' : censor_rating,
            'Movie_Length' : movie_len,
            'Genre_1' : genre_1,
            'Genre_2' : genre_2,
            'Genre_3' : genre_3,
            'Genre_4' : genre_4,
            'Release_Date' : release_date,
            'Story_Summary' : story_summary,
            'Director' : director,
            'Writer_1' : writer_1,
            'Writer_2' : writer_2,
            'Writer_3' : writer_3,
            'Star_1' : star_1,
            'Star_2' : star_2,
            'Star_3' : star_3,
            'Star_4' : star_4,
            'Star_5' : star_5,
            'Plot_Keywords' : plot_keywords,
            'Budget' : budget,
            'Gross_USA' : gross_usa,
            'Cum_Worldwide_Gross' : cum_world_gross,
            'Production_Company' : production_company
            }
        #---------------------------Append rows to dataframes using dictionary
        df = df.append(pd.DataFrame.from_records([movie_dict],columns=movie_dict.keys() ) )
    
    except Exception:
        pass

In [15]:
df=df[column_list]  
df=df.set_index(['Rank'], drop=False) #should be run only once

In [16]:
df.head(10)

,Rank,Movie_name,URL,Release_Year,IMDB_Rating,Metascore,Reviewer_count,Censor_Board_Rating,Movie_Length,Genre_1,...,Star_1,Star_2,Star_3,Star_4,Star_5,Plot_Keywords,Budget,Gross_USA,Cum_Worldwide_Gross,Production_Company
Rank,,,,,,,,,,,,,,,,,,,,,
1,1,The Shawshank Redemption,https://www.imdb.com/title/tt0111161/,1994,9.3,80,"2,260,257",PG,2h 22min,Drama,...,Tim Robbins,Morgan Freeman,Bob Gunton,,,wrongful imprisonment|based on the works of st...,"$25,000,000(estimated)","$28,699,976","$28,815,245",Castle Rock Entertainment
2,2,The Godfather,https://www.imdb.com/title/tt0068646/,1972,9.2,100,"1,559,568",PG,2h 55min,Crime,...,Marlon Brando,Al Pacino,James Caan,,,mafia|crime family|patriarch|organized crime|r...,"$6,000,000(estimated)","$134,966,411","$246,120,974","Paramount Pictures,Alfran Productions"
3,3,The Godfather: Part II,https://www.imdb.com/title/tt0071562/,1974,9.0,90,"1,089,911",PG,3h 22min,Crime,...,Al Pacino,Robert De Niro,Robert Duvall,,,revenge|corrupt politician|bloody body of chil...,"$13,000,000(estimated)","$47,834,595","$48,035,783","Paramount Pictures,The Coppola Company,America..."
4,4,The Dark Knight,https://www.imdb.com/title/tt0468569/,2008,9.0,84,"2,225,638",PG-13,2h 32min,Action,...,Christian Bale,Heath Ledger,Aaron Eckhart,,,dc comics|joker|psychopath|clown|criminal mast...,"$185,000,000(estimated)","$535,234,033","$1,099,094,480","Warner Bros.,Legendary Entertainment,Syncopy"
5,5,12 Angry Men,https://www.imdb.com/title/tt0050083/,1957,8.9,96,"662,177",Not Rated,1h 36min,Crime,...,Henry Fonda,Lee J. Cobb,Martin Balsam,,,jury|dialogue driven|courtroom|single set prod...,"$350,000(estimated)",,$576,Orion-Nova Productions
6,6,Schindler's List,https://www.imdb.com/title/tt0108052/,1993,8.9,94,"1,174,098",PG,3h 15min,Biography,...,Liam Neeson,Ralph Fiennes,Ben Kingsley,,,accountant|champagne|villa|womanizer|soap,"$22,000,000(estimated)","$96,898,818","$322,287,794","Universal Pictures,Amblin Entertainment"
7,7,The Lord of the Rings: The Return of the King,https://www.imdb.com/title/tt0167260/,2003,8.9,94,"1,594,620",PG,3h 21min,Adventure,...,Elijah Wood,Viggo Mortensen,Ian McKellen,,,orc|epic|battle|journey|hobbit,"$94,000,000(estimated)","$377,845,905","$1,141,982,172","New Line Cinema,WingNut Films,The Saul Zaentz ..."
8,8,Pulp Fiction,https://www.imdb.com/title/tt0110912/,1994,8.9,94,"1,768,349",18,2h 34min,Crime,...,John Travolta,Uma Thurman,Samuel L. Jackson,,,nonlinear timeline|overdose|drug overdose|bond...,"$8,000,000(estimated)","$107,928,762","$222,831,519","Miramax,A Band Apart,Jersey Films"
9,9,"Il buono, il brutto, il cattivo",https://www.imdb.com/title/tt0060196/,1966,8.8,90,"667,396",Not Rated,2h 41min,Western,...,Clint Eastwood,Eli Wallach,Lee Van Cleef,,,spaghetti western|shootout|famous opening them...,"$1,200,000(estimated)","$25,100,000","$25,252,481","Produzioni Europee Associate (PEA),Arturo Gonz..."


In [17]:
df.tail(10)

,Rank,Movie_name,URL,Release_Year,IMDB_Rating,Metascore,Reviewer_count,Censor_Board_Rating,Movie_Length,Genre_1,...,Star_1,Star_2,Star_3,Star_4,Star_5,Plot_Keywords,Budget,Gross_USA,Cum_Worldwide_Gross,Production_Company
Rank,,,,,,,,,,,,,,,,,,,,,
240,240,Drishyam,https://www.imdb.com/title/tt4430212/,2015,8.2,Reviews219 user|39 critic,"64,321",Not Rated,2h 43min,Crime,...,Ajay Devgn,Shriya Saran,Tabu,,,pursuit|accident|man beats up woman|cinema|wit...,"$9,400,000(estimated)",,,"Panorama Studios,Rajkumar Theaters Private Lim..."
241,241,The Help,https://www.imdb.com/title/tt1454029/,2011,8.1,62,"416,979",PG-13,2h 26min,Drama,...,Emma Stone,Viola Davis,Octavia Spencer,,,chocolate pie|writer|newspaper columnist|aging...,"$25,000,000(estimated)","$169,708,112","$216,639,112","DreamWorks,Reliance Film & Entertainment,Parti..."
242,242,The Princess Bride,https://www.imdb.com/title/tt0093779/,1987,8.1,77,"382,343",Not Rated,1h 38min,Adventure,...,Cary Elwes,Mandy Patinkin,Robin Wright,,,true love|giant|fairy tale|pirate|princess,"$16,000,000(estimated)","$30,857,814","$30,877,156","Act III Communications,Buttercup Films Ltd.,Th..."
243,243,Contratiempo,https://www.imdb.com/title/tt4857264/,2016,8.1,Reviews380 user|64 critic,"123,038",Not Rated,1h 46min,Crime,...,Mario Casas,Ana Wagener,Jose Coronado,,,repeated scene from a different perspective|gi...,"EUR4,000,000(estimated)",,"$31,191,910","Atresmedia Cine,Think Studio,Nostromo Pictures"
244,244,The Terminator,https://www.imdb.com/title/tt0088247/,1984,8.0,84,"779,421",Not Rated,1h 47min,Action,...,Arnold Schwarzenegger,Linda Hamilton,Michael Biehn,,,robotics|time travel|future war|pig|bacon,"$6,400,000(estimated)","$38,371,200","$78,559,818","Cinema '84,Euro Film Funding,Hemdale"
245,245,Fanny och Alexander,https://www.imdb.com/title/tt0083922/,1982,8.1,100,"55,758",Not Rated,3h 8min,Drama,...,Bertil Guve,Pernilla Allwin,Kristina Adolphson,,,domestic violence|grief|marriage of convenienc...,"$6,000,000(estimated)","$6,783,304","$6,795,771","Cinematograph AB,Svenska Filminstitutet (SFI),..."
246,246,La battaglia di Algeri,https://www.imdb.com/title/tt0058946/,1966,8.1,96,"50,437",Not Rated,2h 1min,Drama,...,Brahim Hadjadj,Jean Martin,Yacef Saadi,,,battle of algiers|algeria|resistance|struggle ...,"$800,000(estimated)","$879,794","$964,028","Casbah Film,Casbah Film,Igor Film"
247,247,PK,https://www.imdb.com/title/tt2338151/,2014,8.1,Reviews583 user|72 critic,"154,922",Not Rated,2h 33min,Comedy,...,Aamir Khan,Anushka Sharma,Sanjay Dutt,,,religion|tv reporter|alien on earth|papadum|mu...,,"$10,616,104","$100,370,104","Rajkumar Hirani Films,Vinod Chopra Productions"
248,248,Aladdin,https://www.imdb.com/title/tt0103639/,1992,8.0,86,"361,270",Not Rated,1h 30min,Animation,...,Scott Weinger,Robin Williams,Linda Larkin,,,arab|genie|genie character|princess|cave,"$28,000,000(estimated)","$217,350,219","$504,050,219","Walt Disney Pictures,Silver Screen Partners IV..."


In [18]:
df.to_csv('IMDB_250_Dataset.csv', index=False)